In [1]:
import os
from crewai import Agent,Crew,Task


In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
# This agent finds out the nodes and relationships and return json
node_extractor = Agent(
    role="Entity-Relationship Extractor",
    goal="Extract entities (nodes) and relationships from a short story as structured JSON.",
    backstory="You are an expert information extractor who maps narratives into graph form.",
    llm="groq/llama-3.3-70b-versatile",
    verbose=True
)

# This agent should generate the query from extracted json
cypher_writer = Agent(
    role="Cypher Query Writer",
    goal="Convert the JSON of nodes and relationships into a Cypher query for Neo4j.",
    backstory="You are a database engineer skilled in writing graph queries.",
    llm="groq/llama-3.3-70b-versatile",
    verbose=True
)


In [4]:
story = """
Alice met Bob in Wonderland. They became friends and visited the castle together.
"""

In [7]:
extract_task = Task(
    description=f"Extract all nodes (characters, places, things) and relationships (actions, connections) from this story as JSON.\nStory:\n{story}",
    expected_output="A JSON object with 'nodes' and 'relationships' arrays describing the story graph.",
    agent=node_extractor
)

In [ ]:
cypher_task = Task(
    description="Take the JSON output from the extractor and write a Cypher query that MERGEs all nodes and relationships into a Neo4j graph database.",
    expected_output="A Cypher query string using MERGE for nodes and relationships.",
    agent=cypher_writer,
    context=[extract_task]  # Pass previous output automatically
)

In [9]:
crew = Crew(
    agents=[node_extractor, cypher_writer],
    tasks=[extract_task, cypher_task],
    verbose=True
)

In [10]:
result = crew.kickoff()



┌───────────────────────────── Execution Traces ──────────────────────────────┐
│                                                                             │
│  🔍 Detailed execution traces are available!                                │
│                                                                             │
│  View insights including:                                                   │
│    • Agent decision-making process                                          │
│    • Task execution flow and timing                                         │
│    • Tool usage details                                                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘


Would you like to view your execution traces? [y/N] (20s timeout): 

In [11]:
print("\n=== Final Cypher Query ===")
print(result)


=== Final Cypher Query ===
```cypher
MERGE (alice:Character {id: 1, name: 'Alice', type: 'character'})
MERGE (bob:Character {id: 2, name: 'Bob', type: 'character'})
MERGE (wonderland:Place {id: 3, name: 'Wonderland', type: 'place'})
MERGE (castle:Place {id: 4, name: 'castle', type: 'place'})

MERGE (alice)-[:MET {id: 1, context: 'in Wonderland'}]->(bob)
MERGE (alice)-[:BECAME_FRIENDS {id: 2}]->(bob)
MERGE (alice)-[:VISITED {id: 3, context: 'with Bob'}]->(castle)
MERGE (bob)-[:VISITED {id: 4, context: 'with Alice'}]->(castle)
MERGE (alice)-[:LOCATED_IN {id: 5}]->(wonderland)
MERGE (bob)-[:LOCATED_IN {id: 6}]->(wonderland)
MERGE (castle)-[:LOCATED_IN {id: 7}]->(wonderland)
```
